In [2]:
!pip install selenium
!pip install scrapy

In [76]:
import numpy as np
import pandas as pd
from glob import glob
from scrapy.selector import Selector
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [77]:
file = "C:/Users/TEEM/Desktop/text data/Movie review URL.xlsx"
dfs=[]
for f in glob(file):
    dfs.append(pd.read_excel(f))
df = pd.concat(dfs, ignore_index=True)

In [78]:
df

,Movie,URL
0,The Shawshank Redemption,https://www.imdb.com/title/tt0111161/reviews/?...
1,The Godfather,https://www.imdb.com/title/tt0068646/reviews/?...
2,The Dark Knight,https://www.imdb.com/title/tt0468569/reviews/?...
3,The Godfather Part II,https://www.imdb.com/title/tt0071562/reviews/?...
4,12 Angry Men,https://www.imdb.com/title/tt0050083/reviews/?...
...,...,...
95,Citizen Kane,https://www.imdb.com/title/tt0033467/reviews/?...
96,M - Eine Stadt sucht einen Mörder,https://www.imdb.com/title/tt0022100/reviews/?...
97,Lawrence of Arabia,https://www.imdb.com/title/tt0056172/reviews/?...
98,North by Northwest,https://www.imdb.com/title/tt0053125/reviews/?...


In [79]:
def call_review(url):
    driver = webdriver.Chrome('chromedriver.exe')
    time.sleep(1)
    driver.get(url)

    time.sleep(1)
    body = driver.find_element(By.CSS_SELECTOR, 'body')
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)
    body.send_keys(Keys.PAGE_DOWN)
    
    review_page(driver)

In [80]:
def review_page(driver):
    sel = Selector(text = driver.page_source)
    # review_counts = sel.css('.lister .header span::text').extract_first().replace(',','').split(' ')[0]
    review_counts = 100
    more_review_pages = int(int(review_counts)/25)
    
    for i in range(more_review_pages):
        try:
            css_selector = 'load-more-trigger'
            driver.find_element(By.ID, css_selector).click()
        except:
            pass
        
    get_review(driver)

In [81]:
def get_review(driver):
    rating_list = []
    review_list = []
    error_url_list = []
    error_msg_list = []

    reviews = driver.find_elements(By.CSS_SELECTOR, 'div.review-container')
    for d in tqdm(reviews):
        try:
            sel2 = Selector(text = d.get_attribute('innerHTML'))
            try:
                rating = sel2.css('.rating-other-user-rating span::text').extract_first()
            except:
                rating = np.NaN
            try:
                review = sel2.css('.text.show-more__control::text').extract()
            except:
                review = np.NaN

            rating_list.append(rating)
            review_list.append(review)

        except Exception as e:
            error_url_list.append(url)
            error_msg_list.append(e)
    
    review_df = pd.DataFrame({
        'Rating':rating_list,
        'Review':review_list,
        })
    print(review_df)
    

In [82]:
# for index, row in df.iterrows():
call_review('https://www.imdb.com/title/tt0111161/reviews/?ref_=tt_ql_urv') #row['URL']

100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 97.14it/s]


   Rating                                             Review
0      10  [The Shawshank Redemption is written and direc...
1      10  [It is no wonder that the film has such a high...
2      10  [I'm trying to save you money; this is the las...
3      10  [This movie is not your ordinary Hollywood fli...
4      10  [In its Oscar year, Shawshank Redemption (writ...
5      10  [One of the finest films made in recent years....
6      10  [Misery and Stand By Me were the best adaptati...
7      10  [I've lost count of the number of times I have...
8      10  [Two imprisoned men (Tim Robbins and Morgan Fr...
9      10  [Based on a novella by Stephen King, this is b...
10     10  [The best movie in history and the best ending...
11   None  [The Shawshank Redemption has great performanc...
12     10  [Well I guess I'm a little late to the party a...
13     10  [Whenever I talk about this movie with my frie...
14     10  [The Shawshank Redemption is without a doubt o...
15   None  [I have never